In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
old_path1 = 'D:/jupyterfile/因子数据/研究因子/'
data1 = [pd.read_csv(old_path1+path,index_col=0) for path in os.listdir(old_path1) if 'csv' in path]
old_path2 = 'D:/jupyterfile/因子数据/控制变量/'
data2 = [pd.read_csv(old_path2+path,index_col=0) for path in os.listdir(old_path2) if 'csv' in path]


In [3]:
#统一时间范围，可手动调整len(data2)
for i in range(len(data2)):
    data2[i]=data2[i].iloc[0:len(data1[0])]
data = data1+data2

In [4]:
#去除极端值
def del_99_1(temp):
    q_99 = temp.quantile(0.99,axis=1)
    q_1 = temp.quantile(0.01,axis=1)
    temp = temp.T
    return temp[(temp>=q_1)&(temp<=q_99)].T
data = [del_99_1(factor) for factor in data]

In [5]:
#将索引改为时间序列
index = pd.DatetimeIndex(data[0].index)
def get_index_column(temp):
    temp.index = index;
    temp.columns = temp.columns.astype('float')
    return temp
data = [get_index_column(factor) for factor in data]

In [6]:
#用列表取出所有列索引，并将每个变量共同拥有的列索引取出
def get_only(list_):
    temp = []
    for factor in list_:
        temp = temp + list(factor.columns)
    return pd.Series(temp)

columns = get_only(data)
columns = columns.value_counts().sort_index()
columns = columns[columns==columns.max()].index

In [7]:
#计算组合收益率，无风险收益为银行利率计算，read时也需要统一时间范围，可手动调整
ret_month = pd.read_csv('D:/jupyterfile/因子数据/股票收益率/ret_month.csv',index_col=0).iloc[0:len(data1[0])]
rf = pd.read_csv('D:/jupyterfile/因子数据/股票收益率/rf.csv',index_col=0).iloc[0:len(data1[0])]

ret_month.index = index
rf.index = index

ret_month.columns = ret_month.columns.astype('float')
ret_month = ret_month[columns]
ret_month = ret_month - rf.values

ret_month = ret_month.shift(-1)

In [8]:
#对所有数据统一化
data = [factor[columns] for factor in data]
data = [ret_month]+data

In [10]:
pool = pd.DataFrame(True,index=index,columns=columns)
#构建股票池，将所有因子共有的值取为True
for factor in data:
    pool = pool&(~factor.isna())

data = [factor[pool] for factor in data]
data = pd.concat([factor.T.stack() for factor in data],axis=1)
data.index.names = ['code','date']
pool

,1.0,2.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,...,688466.0,688505.0,688516.0,688518.0,688520.0,688555.0,688566.0,688588.0,688598.0,688599.0
trade_date,,,,,,,,,,,,,,,,,,,,,
2001-01-31,False,True,False,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2001-02-28,False,True,False,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2001-03-31,False,True,False,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2001-04-30,False,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2001-05-31,False,True,False,True,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31,False,False,True,True,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-09-30,False,False,True,True,True,True,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
2020-10-31,False,False,True,True,True,False,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False


In [11]:
data

0         1         2         3          4  \
code     date                                                            
1.0      2003-09-30 -0.113958  0.008900  1.287230  0.223160  23.604888   
         2003-10-31  0.027849  0.029753  1.272123  0.251398  23.485741   
         2003-11-30  0.013878  0.014767  1.289896  0.244198  23.514798   
         2003-12-31  0.088847  0.013672  1.337315  0.239453  23.530192   
         2004-01-31  0.114744  0.011804  1.330531  0.243387  23.616812   
...                       ...       ...       ...       ...        ...   
603999.0 2020-07-31  0.025908  0.242586  1.318162  0.453179  22.063223   
         2020-08-31 -0.152489  0.219438  1.255334  0.441201  22.090010   
         2020-09-30 -0.032383  0.023008  1.180690  0.529022  21.926022   
         2020-10-31  0.123759 -0.011491  1.154601  0.546026  21.894385   
         2020-11-30 -0.125051  0.076346  1.146227  0.485356  22.012168   

                            5         6  
code     date                            
1.0      2003-09-30  0.015919 -0.249819  
         2003-10-31  0.025028 -0.280181  
         2003-11-30  0.049438 -0.316147  
         2003-12-31  0.034347 -0.187932  
         2004-01-31  0.045909 -0.256170  
...                       ...       ...  
603999.0 2020-07-31  0.049322  0.122639  
         2020-08-31  0.034860  0.185846  
         2020-09-30  0.028317  0.211573  
         2020-10-31  0.024180 -0.008526  
         2020-11-30  0.055235 -0.114921  

[333461 rows x 7 columns]

In [12]:
lscolumns=[]
for i in os.listdir(old_path2):
    i=i[:-4]
    lscolumns.append(i)


In [13]:
data.columns = ['ret_month','tail_risk']+lscolumns[1:]
data

ret_month  tail_risk      beta        bm    log_cap  \
code     date                                                              
1.0      2003-09-30  -0.113958   0.008900  1.287230  0.223160  23.604888   
         2003-10-31   0.027849   0.029753  1.272123  0.251398  23.485741   
         2003-11-30   0.013878   0.014767  1.289896  0.244198  23.514798   
         2003-12-31   0.088847   0.013672  1.337315  0.239453  23.530192   
         2004-01-31   0.114744   0.011804  1.330531  0.243387  23.616812   
...                        ...        ...       ...       ...        ...   
603999.0 2020-07-31   0.025908   0.242586  1.318162  0.453179  22.063223   
         2020-08-31  -0.152489   0.219438  1.255334  0.441201  22.090010   
         2020-09-30  -0.032383   0.023008  1.180690  0.529022  21.926022   
         2020-10-31   0.123759  -0.011491  1.154601  0.546026  21.894385   
         2020-11-30  -0.125051   0.076346  1.146227  0.485356  22.012168   

                          Max       mom  
code     date                            
1.0      2003-09-30  0.015919 -0.249819  
         2003-10-31  0.025028 -0.280181  
         2003-11-30  0.049438 -0.316147  
         2003-12-31  0.034347 -0.187932  
         2004-01-31  0.045909 -0.256170  
...                       ...       ...  
603999.0 2020-07-31  0.049322  0.122639  
         2020-08-31  0.034860  0.185846  
         2020-09-30  0.028317  0.211573  
         2020-10-31  0.024180 -0.008526  
         2020-11-30  0.055235 -0.114921  

[333461 rows x 7 columns]